In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import time

def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [2]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [3]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(0)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper
Gripper activated


In [4]:
home_pos = [-0.19171785391383447,
 0.13627313847997366,
 0.22530939672205824,
 -2.216400117559973,
 -2.214550992098179,
 -0.011655385945140848]

home_joints = [3.1417369842529297,
 -0.7853723925403138,
 -2.3560631275177,
 -1.570796628991598,
 1.5707793235778809,
 0.0006903879693709314]

front_of_incubator_pos = [-0.18132849937991355,
 -0.3679708228594073,
 0.720661961195772,
 1.2204624721671815,
 -1.2040970374232316,
 1.2241694975948347]## new


front_of_incubator_joints = [4.383610725402832,
 -1.281290666466095,
 -1.1723852157592773,
 -3.8264199695982875,
 4.360137939453125,
 1.583448886871338]

over_changer_joints = [3.339607000350952,
 -2.292309423486227,
 -1.422972559928894,
 -2.5316902599730433,
 0.2237834930419922,
 4.67471981048584]

front_of_new_flasks_pos = [0.2950583758025984,
 0.3867662704396315,
 0.045458652971989186,
 -1.2186923157725582,
 -1.1848882312252083,
 -1.218559178432022]

turned_to_flask_pos = [0.12732424856627178,
 0.19778719494662886,
 0.22530185297353472,
 -3.139800345487405,
 -0.07107586690583619,
 -0.01769962787541179]

turned_to_flask_joints = [0.891446590423584,
 -2.110570570031637,
 -2.4078848361968994,
 -1.7478038273253382,
 4.060787200927734,
 1.5948678255081177]

incubator_closing_joints = [2.482529401779175,
 -1.2133799952319642,
 -1.1805007457733154,
 -3.884266277352804,
 4.1642889976501465,
 1.5839492082595825]

front_of_miscroscope_pos = [-0.41388050689659883,
 0.1750650772622419,
 0.08218153855092553,
 0.0406564345763103,
 -1.570776637193879,
 0.019654881719202173]

microscope_pos_1 = [-0.6023525432650326,
 0.1980722952723789,
 0.08215312741288205,
 0.04068837916844658,
 -1.5708440058334778,
 0.019607396781294046]

front_of_decapper_pos = [-0.41390485457494364,
 0.39929584431254445,
 0.16074269025000604,
 0.040680734747890156,
 -1.5707670056970375,
 0.01974281965873683]

above_flask_box = [-0.5808590668495738,
 0.46350849977019337,
 0.22695919785317129,
 1.225880090768996,
 -1.2353631461828702,
 -1.1937109628573248]

at_cap = [-0.6202925767109608,
 0.4493540106520409,
 0.2207983835751558,
 0.010282978008100973,
 -3.107402683234777,
 -0.4411787994503421]

above_cap = [-0.6203323667337821,
 0.44931542104142785,
 0.45085035480129525,
 -2.6840097665737828,
 1.033303485109148,
 0.15225961997584497]

above_cap_joints = [2.772878885269165,
 -2.24400057415151,
 -0.43866968154907227,
 -1.9212991199889125,
 1.3188759088516235,
 -4.364126507435934]



In [5]:
def pick_up_flask(position): #lowest: position 5, highest: position 0
    gripper.open()
    #rtde_c.moveL(home_pos, 0.1, 0.1)
    rtde_c.moveJ(home_joints, 1.5, 0.8)
    rtde_c.moveJ(front_of_incubator_joints, 0.4, 0.2) ##highest position, better than moveL

    #move down for different positions
    pick_up_pos = front_of_incubator_pos.copy()
    pick_up_pos[2] -= position*0.09
    rtde_c.moveL(pick_up_pos, 0.1, 0.1)
    
    pick_up_pos[1] -= 0.244593
    rtde_c.moveL(pick_up_pos, 0.05, 0.1)
    gripper.set_force(10)  # from 0 to 100 %
    gripper.close()

In [6]:
def remove_from_incubator():
    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] += 0.0084224
    rtde_c.moveL(current_pos, 0.05, 0.1)
    current_pos[1] += 0.32
    rtde_c.moveL(current_pos, 0.3, 0.2)

In [7]:
def bring_to_incubator_closing_pos():
    rtde_c.moveJ(incubator_closing_joints, 1.5, 0.8)
    # wait 5 seconds for the incubator to close
    time.sleep(5)

In [8]:
def bring_to_microscope():
    rtde_c.moveL(front_of_miscroscope_pos, 0.2, 0.2)
    rtde_c.moveL(microscope_pos_1, 0.1, 0.1)
    # wait 2 seconds for the microscope to analyze
    time.sleep(2)
    current_pos = rtde_r.getActualTCPPose()
    current_pos[0] -= 0.01
    rtde_c.moveL(current_pos, 0.05, 0.1)
    # wait 2 seconds for the microscope to analyze
    time.sleep(2)
    current_pos[0] += 0.02
    rtde_c.moveL(current_pos, 0.05, 0.1)
    # wait 2 seconds for the microscope to analyze
    time.sleep(2)
    rtde_c.moveL(front_of_miscroscope_pos, 0.1, 0.1)

In [9]:
def place_in_decapper_pos():
    rtde_c.moveL(front_of_decapper_pos, 0.2, 0.2)
    rtde_c.moveL(above_flask_box, 0.2, 0.2)
    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] -= 0.14126
    rtde_c.moveL(current_pos, 0.02, 0.1)
    gripper.open()
    current_pos[2] += 0.25
    rtde_c.moveL(current_pos, 0.05, 0.1)
    #rtde_c.moveL(front_of_decapper_pos, 0.1, 0.1)


In [10]:
def decap_flask():
    rtde_c.moveJ(above_cap_joints, 0.3, 0.3)
    gripper.open()
    rtde_c.moveL(at_cap, 0.1, 0.1)
    gripper.set_force(25)
    gripper.close()
    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] += 0.015
    rtde_c.moveL(current_pos, 0.1, 0.1)

    current_joints = rtde_r.getActualQ()
    current_joints[5] -= 1.2
    rtde_c.moveJ(current_joints, 0.1, 0.1)

    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] += 0.01
    rtde_c.moveL(current_pos, 0.1, 0.1)

    current_joints = rtde_r.getActualQ()
    current_joints[5] -= 1.2
    rtde_c.moveJ(current_joints, 0.1, 0.1)

    current_pos = rtde_r.getActualTCPPose()
    current_pos[2] += 0.05
    rtde_c.moveL(current_pos, 0.1, 0.1)

    rtde_c.moveJ(home_joints, 0.5, 0.5)

In [11]:
pick_up_flask(0)
remove_from_incubator()
bring_to_incubator_closing_pos()
bring_to_microscope()
place_in_decapper_pos()
decap_flask()

RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!
RTDEControlInterface: RTDE control script is not running!


KeyboardInterrupt: 

: 

In [26]:
current_pos = rtde_r.getActualTCPPose()
current_pos

[-0.6203323667337821,
 0.44931542104142785,
 0.25085035480129525,
 -2.6840097665737828,
 1.033303485109148,
 0.15225961997584497]

In [7]:
init_q = rtde_r.getActualQ()
init_q

[2.772878885269165,
 -2.24400057415151,
 -0.43866968154907227,
 -1.9212991199889125,
 1.3188759088516235,
 -4.364126507435934]